<a href="https://colab.research.google.com/github/FrogyKing/MBAIA/blob/main/Principales_tecnologias_de_la_IA/sesion4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea NLP

In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open("conocimientos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
preguntas = [item["pregunta"] for item in data]
respuestas = [item["respuesta"] for item in data]

In [4]:
vectorizador = TfidfVectorizer()
X = vectorizador.fit_transform(preguntas)

In [50]:
def responder(pregunta_usuario):
    vector_usuario = vectorizador.transform([pregunta_usuario])
    similitudes = cosine_similarity(vector_usuario, X)
    idx = similitudes.argmax()
 # Umbral de confianza
    if similitudes[0, idx] < 0.2:
        return "Lo siento, solo puedo responder preguntas de Historia del Perú."
    return respuestas[idx]

## P1: Ampliar dataset

El dataset se amplió con 100 preguntas y respuestas en total

In [6]:
len(data)

100

## P2: Probar con preguntas ambiguas o mal escritas

### Sin preprocesamiento

#### Ejemplo 1

In [51]:
q_correcta = "Cuando fue la independencia del peru"
q_incorrecta = "Cuando fue la independensia del peru"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
La independencia del Perú fue proclamada por José de San Martín el 28 de julio de 1821.
Versión incorrecta
La esclavitud fue abolida en el Perú el 3 de diciembre de 1854 por el presidente Ramón Castilla.


#### Ejemplo 2

In [52]:
q_correcta = "Quien fue bolognesi"
q_incorrecta = "Quien fue bologneci"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.
Versión incorrecta
Atahualpa fue el último emperador inca, capturado y ejecutado por los conquistadores españoles en 1533.


Los respuestas son muy sensibles a errores ortograficos debido a que TFidVectorizer trabaja solo con coincidencias exactas de palabras.

Se podría corregir con un corrector ortográfico o usando embedding semánticos

### Con preprocesamiento usando embeddings

In [11]:
%pip install -q --upgrade google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00


In [28]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.1 MB/s eta 0:00:00


In [29]:
from google.colab import userdata
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
llm_model = "gemini-2.5-flash"
api_key = userdata.get('GOOGLE_API_KEY')

In [30]:
docs = [
    Document(page_content=d["pregunta"], metadata={"respuesta": d["respuesta"]})
    for d in data
]

In [31]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=api_key)

In [34]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_historia_peru")

In [54]:
def responder(pregunta_usuario: str) -> str:
    resultados = db.similarity_search(pregunta_usuario, k=1)

    if not resultados:
        return "Lo siento, solo puedo responder preguntas de Historia del Perú."

    mejor_match = resultados[0]
    respuesta = mejor_match.metadata.get("respuesta", "No se encontró una respuesta.")

    return respuesta

#### Ejemplo 1

In [55]:
q_correcta = "Cuando fue la independencia del peru"
q_incorrecta = "Cuando fue la independensia del peru"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
José de San Martín proclamó la independencia del Perú el 28 de julio de 1821 en Lima.
Versión incorrecta
José de San Martín proclamó la independencia del Perú el 28 de julio de 1821 en Lima.


#### Ejemplo 2

In [56]:
q_correcta = "Quien fue bolognesi"
q_incorrecta = "Quien fue bologneci"
print("Versión correcta")
print(responder(q_correcta))
print("Versión incorrecta")
print(responder(q_incorrecta))

Versión correcta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.
Versión incorrecta
Francisco Bolognesi fue un héroe nacional que defendió Arica en 1880 diciendo: 'Tengo deberes sagrados que cumplir'.


# P3: Extender el bot a bsuqueda de fuentes de inforamción en la web